<a href="https://colab.research.google.com/github/arjunvooturi/Captial_placement/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 137.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Load the Hugging Face job descriptions dataset
dataset = load_dataset("jacob-hugging-face/job-descriptions")

# Fetch 10-15 job descriptions
num_samples = 15
job_descriptions = dataset["train"]["job_description"][:num_samples]

# Print the job descriptions
for i, description in enumerate(job_descriptions):
    print(f"Job Description {i + 1}:\n")
    print(description)
    print("\n" + "=" * 50 + "\n")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Job Description 1:

minimum qualifications
bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles
preferred qualifications
 years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills
about the job
as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organi

In [3]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/Resume.csv')
data.info()

In [ ]:
col = ['Resume_html', 'Category']
df = data.drop(columns=col)

In [ ]:
df

In [ ]:
df.head()

In [ ]:
!pip install spacy


In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import pandas as pd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


In [ ]:
def extract_skills(text):
    doc = nlp(text)
    skills = []
    for ent in doc.ents:
        if ent.label_ == "ORG":
            skills.append(ent.text)
    return skills


In [ ]:
df["Extracted_Skills"] = df["Resume_str"].apply(extract_skills)

In [ ]:
print(df[["ID", "Extracted_Skills"]])


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [ ]:
pip install transformers scikit-learn numpy

In [ ]:
import pandas as pd

# Load your resume dataset into a DataFrame
df = pd.read_csv('/content/Resume.csv')

# Extract resume text as a list of strings
resumes = df['Resume_str'].tolist()

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load the pretrained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def tokenize_and_embed(texts):
    # Tokenize the texts
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

    with torch.no_grad():
        outputs = model(**tokens)

    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Tokenize and embed resume data and job descriptions in batches
resume_embeddings = tokenize_and_embed(resumes)
job_embeddings = tokenize_and_embed(job_descriptions)

# Calculate Cosine Similarity
similarity_scores = cosine_similarity(job_embeddings, resume_embeddings)

# Rank Resumes
ranked_resumes = np.argsort(similarity_scores, axis=1)[:, ::-1]

# List Top 5 Resumes for Each Job Description
top_5_resumes = []
for i, ranked_indices in enumerate(ranked_resumes):
    top_5_indices = ranked_indices[:5]
    top_5_resumes.append([(resumes[j], similarity_scores[i, j]) for j in top_5_indices])
